"""
Script to download glider (slocum) data from the C2/API web services at NOC MARS.  

1. Works locally on the downloaded raw netcdf files.
2. Calculate MLD
3. Calculate oxygen


Runs offline, using the netcdf files created in update_raw_data.py
"""

In [6]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import os
import glob
import datetime as dt
import requests
import json
from io import StringIO
import ast # To handle the string conversion when loading json filee
from pathlib import Path
# Own packages of code
from setdir import *
from parseglider import *
from plotglider import *
from calc_oxy import *
from scipy.interpolate import interp1d

In [2]:
# Choice of grid interval (pressure in dbar)
dp=10

## CHANGE TO A CONFIG FILE WITH USER DEFINABLE PARAMETERS
# Slocum gliders: A dictionary with the key as the serial number ('unit_398') 
# and then the plain text name, "Churchill"
glider_names = {
    'unit_398': 'Churchill',
    'unit_409': 'Grease',
}

sensor_sn = {
    'unit_398': {"optode SN": "232"},
    'unit_409': {"optode SN": "268"},
}
# Dictionary keys MUST match the serial number format used in the API.  

# List of glider serial numbers for API
unit_list = [(k) for k in glider_names.keys()]

In [3]:
#--------------------------------------------------------------
# DATA PROCESSING:
# - Calculate oxygen
#
# Save new files to 01-data/03-interim/ as
#   UNIT_data_YYYYMMDD.nc for the full data as a vector
#   UNIT_bin10m.nc for the rough gridded data.
#--------------------------------------------------------------
for uname in unit_list:
    fname = uname+'*_data.nc'
    
    # Extract a list with the names of existing raw data files
    existing_files = glob.glob(cat_interim_path(fname))
    
    # Check whether there are any files
    if len(existing_files) > 0:
        # Extract the most recent filename
        existing_files = sorted(existing_files)
        latest_file = existing_files[-1]
        
        # Open the dataset
        data_ds = xr.open_dataset(latest_file)
        
        #--------------------------------------------------------------
        # Calculate oxygen (maybe this should be elsewhere)
        # but I need it before gridding (or need a function to grid
        # individual extra variables)
        #--------------------------------------------------------------
        sensorsn1 = sensor_sn[uname]
        data_ds = data_ds.assign_attrs(sensorsn1)
        data_ds = calc_o2conc_cal(data_ds)
        print('Doing it')
        fname2 = latest_file[0:-3]+'_o2.nc'

        # Add oxygen to the file and save it to the same path (interim)
        # Save updated with oxygen
        data_ds.to_netcdf(fname2, mode='w')
        data_ds.close()
        

Doing it
Doing it


In [23]:
glider_names = {
    'unit_409': 'Grease',
}


glider_names = {
    'unit_398': 'Churchill',
}

unit_list = [(k) for k in glider_names.keys()]



In [24]:
for uname in unit_list:
    fname = uname+'*_data_o2.nc'
    
    # Extract a list with the names of existing interim data files
    existing_files = glob.glob(cat_interim_path(fname))
    
    # Check whether there are any files
    if len(existing_files) > 0:
        # Extract the most recent filename
        existing_files = sorted(existing_files)
        latest_file = existing_files[-1]
        
        # Open the dataset
        data_ds = xr.open_dataset(latest_file)
         
        if 0:
            # Check whether a gridded file has already been created
            # Not yet implemented
            proc_files = glob.glob(cat_interim_path(fname))
            if not len(proc_files) > 0:
                print('No processed files for that glider')
   
        #--------------------------------------------------------------
        # Grid data onto a regular pressure grid (intervals given by dp)
        # - Grid data into a 2d matrix against profile index & pressure grid 
        #    NOTE: Gridding is rough and *not* science quality
        # Saves output to 01-data/03-processed/*_bin10m.nc
        #--------------------------------------------------------------
        grid_ds = bin_dp(data_ds, data_ds.attrs['Serial number'], dp)
        
        #-------------------------
        # Add extra vector coordinates
        #--------------------
        mtime = grid_ds.time.mean(dim='pressure').values
        mlon = grid_ds.m_lon.mean(dim='pressure').values
        mlat = grid_ds.m_lat.mean(dim='pressure').values

        # Interpolate over lat and long values
        divenum = grid_ds.divenum.values

        # Lon
        idxnan = (~np.isnan(mlon))
        divenum_nonnan = divenum[idxnan]
        mlon_nonnan = mlon[idxnan]
        flon = interp1d(divenum_nonnan, mlon_nonnan,
                        kind='linear', fill_value="extrapolate")
        mlon_full = flon(divenum)

        # Lat
        idxnan = (~np.isnan(mlat))
        divenum_nonnan = divenum[idxnan]
        mlat_nonnan = mlat[idxnan]
        flat = interp1d(divenum_nonnan,mlat_nonnan,
                        kind='linear', fill_value="extrapolate")
        mlat_full = flat(divenum)

        # Calculate distances from the interpolated lat/lon positions
        dist_km = gsw.distance(mlat_full, mlon_full, 0, axis=0)/1000
        dist_km_pad = np.append(0, dist_km)
        # Cumsum is a problem, need to do something about NaN?
        dist_along_track = np.cumsum(dist_km_pad)

        # Create data array versions
        DAT_2 = xr.DataArray(dist_along_track, 
                             coords={"divenum": grid_ds.divenum},
                             attrs=dict(long_name="Distance", units="km"))
        TIME_2 = xr.DataArray(mtime, 
                              coords={"divenum": grid_ds.divenum},
                             attrs=dict(long_name="Date"))
        LAT_2 = xr.DataArray(mlat_full, 
                             coords={"divenum": grid_ds.divenum},
                            attrs=dict(long_name="Latitude"))
        LON_2 = xr.DataArray(mlon_full, 
                             coords={"divenum": grid_ds.divenum},
                            attrs=dict(long_name="Longitude"))



        grid_ds["dist_along_track"] = DAT_2
        grid_ds["timevec"] = TIME_2
        grid_ds["lonvec"] = LON_2
        grid_ds["latvec"] = LAT_2




        # Change the variables to coordinates
        grid_ds = grid_ds.set_coords(['dist_along_track','timevec',
                                      'lonvec','latvec'])

        #-------------------------------------------------
        # Save gridded    
        #-------------------------------------------------
        data_ds.close()

        # Filename
        uname = data_ds.attrs['Serial number']
        maxtimestr = data_ds.attrs['End Time']
        outfile = uname+'_'+maxtimestr+'_bin10m.nc'
        print(outfile)
        grid_ds.to_netcdf(cat_proc_path(outfile), mode='w')
        grid_ds.close()

        

unit_398_20220308_bin10m.nc


In [1]:
tmp=grid398.time.mean(dim='pressure')
grid398["timevec"] = tmp

grid398

NameError: name 'grid398' is not defined

In [24]:
presname = 'pressure_dbar'
idxname = 'profile_index'
u1 = data_ds
unitname='unit_409'
dp=10

# Create a temporary time variable
u1 = u1.assign(timevar=u1.time.astype('float'))

# Extract an array of pressure and profile_index values
pres = u1[presname].values
prof_idx = u1[idxname].values

# Compute the bins, evenly spaced between the surface and 1000m
# ISSUE: Note the hardcoding of maximum pressure
pmin = 0
pmax = 1000
nbins = int(round((pmax-pmin)/dp))
bins10m = np.linspace(pmin, pmax, nbins+1)
pres10m = np.linspace(pmin+dp/2, pmax-dp/2, nbins)

# Get a unique list of all the dive numbers (whole *.0 - dive, and half *.5 - climb)
divenum_vec = np.unique(u1[idxname].values)
# Remove the nan values
divenum_vec = divenum_vec[~np.isnan(divenum_vec)]

# Updated to operate on all variables in the data array
varlist = list(u1.keys())
varlist.remove(idxname)
varlist.remove(presname)


In [29]:
# Initialise an empty dictionary of variables
myvars = dict()

counter = 0
for i in divenum_vec:
    # Subselect the xarray dataset for the dive or climb indices only
    u11 = u1.where(u1[idxname]==i, drop=True)

    if len(u11["time"])>1:
        # Make sure there are non-NaN values
         # Calculate the median of values within the bin
        ddive = u11.groupby_bins(presname,bins10m, squeeze=False).mean()

In [26]:
print(i)

388.5


In [28]:
len(u11["time"])

1